# WorkflowSession

> Namespaced workflow state management within FastHTML sessions.

In [ ]:
#| default_exp core.workflow_session

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Dict, Any

## WorkflowSession Class

The `WorkflowSession` class provides a scoped view of FastHTML session data for a specific workflow. This prevents key collisions when multiple workflows are active in the same application.

Each workflow gets its own namespace within the session, making it easy to manage multi-step processes without worrying about overwriting data from other workflows.

In [ ]:
#| export
class WorkflowSession:
    """Manage workflow state within FastHTML sessions with proper namespacing."""

    def __init__(
        self, 
        sess:Any, # FastHTML session object (from route handler parameter)
        workflow_id:str # Unique identifier for this workflow (e.g., "registration", "checkout")
    ):
        """Initialize workflow session wrapper."""
        self.sess = sess
        self.workflow_id = workflow_id
        self._prefix = f"workflow_{workflow_id}_"

    def _make_key(
        self, 
        key:str # Key name within workflow namespace
    ) -> str: # Namespaced session key
        """Create namespaced session key."""
        return f"{self._prefix}{key}"

    def set(
        self, 
        key:str, # Key name within this workflow namespace
        value:Any # Value to store (must be JSON-serializable for session storage)
    ) -> None:
        """Store value in workflow session."""
        self.sess[self._make_key(key)] = value

    def get(
        self, 
        key:str, # Key name within this workflow namespace
        default:Any=None # Default value if key not found
    ) -> Any: # Stored value or default
        """Retrieve value from workflow session."""
        return self.sess.get(self._make_key(key), default)

    def delete(
        self, 
        key:str # Key name within this workflow namespace
    ) -> None:
        """Remove value from workflow session."""
        full_key = self._make_key(key)
        if full_key in self.sess:
            del self.sess[full_key]

    def clear(self) -> None:
        """Clear all data for this workflow from session."""
        keys_to_delete = [k for k in self.sess.keys() if k.startswith(self._prefix)]
        for key in keys_to_delete:
            del self.sess[key]

    def get_all(self) -> Dict[str, Any]: # Dictionary mapping original keys to their values
        """Get all workflow data as a dictionary with original (unprefixed) keys."""
        result = {}
        prefix_len = len(self._prefix)
        for key, value in self.sess.items():
            if key.startswith(self._prefix):
                original_key = key[prefix_len:]
                result[original_key] = value
        return result

    def has(
        self, 
        key:str # Key name within this workflow namespace
    ) -> bool: # True if key exists, False otherwise
        """Check if key exists in workflow session."""
        return self._make_key(key) in self.sess

## Usage Examples

These examples demonstrate how to use `WorkflowSession` with a mock session object. In a real FastHTML application, the `sess` parameter is automatically provided by the framework.

In [ ]:
# Create a mock session (in FastHTML, this is provided automatically)
mock_session = {}

# Create workflow session for a registration workflow
workflow = WorkflowSession(mock_session, "registration")
workflow

<__main__.WorkflowSession>

In [ ]:
# Store some values
workflow.set("email", "user@example.com")
workflow.set("username", "john_doe")
workflow.set("step", 2)

# Show the underlying session (note the prefixed keys)
mock_session

{'workflow_registration_email': 'user@example.com',
 'workflow_registration_username': 'john_doe',
 'workflow_registration_step': 2}

In [ ]:
# Retrieve values
email = workflow.get("email")
username = workflow.get("username")
print(f"Email: {email}")
print(f"Username: {username}")

Email: user@example.com
Username: john_doe


In [ ]:
# Get all workflow data
all_data = workflow.get_all()
print("All workflow data:")
all_data

All workflow data:


{'email': 'user@example.com', 'username': 'john_doe', 'step': 2}

In [ ]:
# Check if key exists
has_email = workflow.has("email")
has_password = workflow.has("password")
print(f"Has email: {has_email}")
print(f"Has password: {has_password}")

Has email: True
Has password: False


In [ ]:
# Create a second workflow in the same session - no collision!
checkout_workflow = WorkflowSession(mock_session, "checkout")
checkout_workflow.set("step", 1)  # Same key name, different workflow
checkout_workflow.set("cart_total", 99.99)

print(f"Registration step: {workflow.get('step')}")
print(f"Checkout step: {checkout_workflow.get('step')}")
print(f"\nSession now contains both workflows:")
mock_session

Registration step: 2
Checkout step: 1

Session now contains both workflows:


{'workflow_registration_email': 'user@example.com',
 'workflow_registration_username': 'john_doe',
 'workflow_registration_step': 2,
 'workflow_checkout_step': 1,
 'workflow_checkout_cart_total': 99.99}

In [ ]:
# Delete a specific key
workflow.delete("step")
print(f"Step after deletion: {workflow.get('step', 'Not found')}")
workflow.get_all()

Step after deletion: Not found


{'email': 'user@example.com', 'username': 'john_doe'}

In [ ]:
# Clear all workflow data
workflow.clear()
print("After clearing registration workflow:")
print(f"Registration data: {workflow.get_all()}")
print(f"Checkout data: {checkout_workflow.get_all()}")
print(f"\nSession (checkout workflow still intact):")
mock_session

After clearing registration workflow:
Registration data: {}
Checkout data: {'step': 1, 'cart_total': 99.99}

Session (checkout workflow still intact):


{'workflow_checkout_step': 1, 'workflow_checkout_cart_total': 99.99}

In [ ]:
# Using with default values
empty_workflow = WorkflowSession({}, "empty")
value = empty_workflow.get("nonexistent", "default_value")
print(f"Value with default: {value}")

Value with default: default_value


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()